<center>
    <h1>The Use and Abuse of Random Numbers</h1>
    <h2>Andres Rios-Tascon</h2>
    <h4>CoDaS-HEP 2024</h4>
    <img src="images/PU_lockup.png" style="height:50px;"/>&nbsp&nbsp&nbsp&nbsp&nbsp<img src="images/Iris-hep-4-no-long-name.png" style="height:50px;"/>
    <br/>
    <font size="4">(Based on material by David Lange, Dan Riley, and Tim Mattson)</font>
</center>

## Takeaway message

There are pitfalls and limitations when using random number generators that you need to be aware of.

## Lesson plan

- Primer on Random Number Generators (RNGs)
- An RNG example: Linear Congruential Generator (LCG)
- Exercise 1
- 3 common pitfalls (and how to avoid them)
- Exercise 2
- Summary

# Random Number Generator (RNG)

- It is a tool that generates a sequence of numbers (with some distribution) where each member has no discernible correlation with the others.

- It is not possible prove that a sequence is random. We can only prove that one is not.

## RNGs are used everywhere

- In videogames, introducing variability to make them more interesting.

- In cryptography, to generate encryption keys to keep information and communications secure.
 

- In HEP:
  - Monte Carlo physics generators (e.g. `Pythia`)
  - Monte Carlo detector simulation (e.g. `Geant4`)
  - Monte Carlo digitization (e.g. electronics simulation)
  - Statistical analysis (e.g. significance tests, importance sampling)
  - ...

## "True" random number generators are tricky

Computers are deterministic machines, so one needs an external entropy source.

- User input
  - e.g. tracking your mouse movement

- Physical noise
  - Intel developed an RNG that used thermal noise
  - <https://random.org> uses atmospheric noise

- Some other creative approach
  -  Cloudflare uses a "Wall of Entropy" (Silicon Graphics previously did too)

<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/Lava_lamp_wall_at_Cloudflare_office_-2.jpg/1280px-Lava_lamp_wall_at_Cloudflare_office_-2.jpg" style="height:400px;"/><br/>
    <font size="2">Image from Wikipedia</font>
</center>

These are generally not computationally affordable on the scale we need them. For example a the detector simulation of single TTbar event through the CMS detector takes of order **2 million random numbers**.

## Pseudo-Random Number Generator

- It is a tool that algorithmically computes a sequence of "random-looking" numbers.

- Have some useful properties:
  - Can be faster than "true" RNGs
  - Are reproducible
  - Have minimal hardware requirements
  - Have mechanisms to support parallelism

There are lots of ways to create a not-so-good random number generator. Creating a good or "good enough" random number generator is hard. Leave that go people working specifically on that.

## An RNG example: Linear Congruential Generator (LCG)

- Linear Congruential Generators (LCGs) are fast and simple, and were widely used.

- The sequence of numbers is generated with the following formula.
$$X_{n+1} = (aX_n + c) \text{ mod } m$$
With modulus $m$, multiplier $a$, increment $c$, starting value (or "seed") $X_0$.

- Choice of constants is critical! If the constants are chosen carefully, the sequence length, or "period" is set by the modulus.

- There are standard collections of reasonable values. Many naive RNG implementations are still LCGs (including many default C libraries).

## Let's look at a simple Python implementation

<p style="color:red;">Note: This is for demonstration purposes. Always use a library with good RNGs for your projects.</p>

In [ ]:
import numpy as np
from numpy.typing import NDArray
# If you're not using Binder and don't have numba, just comment out everything from here to "class"
from numba.experimental import jitclass
from numba import int64

spec = [ ('multiplier', int64), ('addend', int64), ('modulus', int64), ('seed', int64), ('last', int64) ]
@jitclass(spec)
class LCG:
    def __init__(
        self,
        # We use "good" default values by default
        seed: int = 1,
        multiplier: int = 1_366,
        addend: int = 150_889,
        modulus: int = 714_025
    ) -> None:
        """Initialize an LCG instance"""
        self.multiplier = multiplier
        self.addend = addend
        self.modulus = modulus
        self.last = seed

    def rand(self) -> float:
        """Return a single random number between 0 and 1"""
        self.last = (self.multiplier * self.last + self.addend) % self.modulus
        return self.last/self.modulus

    def randv(self, size: int = 1) -> NDArray[np.float64]:
        """Return a numpy array of random numbers between 0 and 1"""
        v = np.empty(size, dtype=np.float64)
        for i in range(size):
            v[i] = self.rand()
        return v

## Let's try it out with our own parameters

In [ ]:
my_lcg = LCG(seed=7, multiplier=7, addend=7, modulus=10)

In [ ]:
print(my_lcg.rand())
print(my_lcg.rand())
print(my_lcg.rand())

Seems pretty random so far...

In [ ]:
my_lcg.randv(10)

Oh no... that's bad!

We would have expected a period of 10, but the period is 4. We're starting to see that it's tricky to get it right.

## Let's make a new instance with the default parameters

In [ ]:
my_lcg = LCG()
my_lcg.randv(50)

Now it looks a lot better!

## Exercise 1

You've computed $\pi$ quite a few times this past week.

Let's do it again, but now with RNGs!

**Exercise:** Write a function that computes $\pi$ using the LCG from above.

Hint: Think about throwing darts on something like this.

<center>
    <img src="images/pi_random.svg" style="height:300px;"/><br/>
</center>

In [ ]:
def calc_pi(rng, num_trials: int) -> float:
    """Calculate PI with the given RNG"""
    # Your code goes here

<details>
<summary>
Answer (Don't look until you're done)
</summary>
Here are two possible solutions.<br/><br/>

This one is faster since it uses more compiled code.
<pre><code>
def calc_pi(rng, num_trials: int) -> float:
    """Calculate PI with the given RNG"""
    x = rng.randv(num_trials)
    y = rng.randv(num_trials)

    in_circle = np.sum(x**2 + y**2 <= 1.0)

    return 4*(in_circle/num_trials) 
</code></pre>

<br/>
<br/>
This one uses less memory.
<pre><code>
def calc_pi(rng, num_trials: int) -> float:
    """Calculate PI with the given RNG"""

    in_circle = sum((lcg.rand()**2 + lcg.rand()**2 <= 1.0 for _ in range(num_trials)))

    return 4*(in_circle/num_trials) 
</code></pre>
</details>

## Let's now try using it to see if it works

In [ ]:
calc_pi(my_lcg, 10_000)

## Let's see how it works as we increase the number of trials

In [ ]:
powers=8
iterations=10

for decade in range(powers) :
    trials = 10**decade
    pis=np.empty(iterations, dtype=np.float64)
    for i in range(iterations):
        pis[i] = calc_pi(my_lcg, trials)
    pi = np.average(pis)
    avg_err = np.average(np.abs(pis - np.pi))
    std = np.std(pis)
    print(f"{trials:8d} trials:", end=" ")
    print(f"pi = {pi:1.7f}", end=", ")
    print(f"avg err = {avg_err:1.7f}", end=", ")
    print(f"std = {std:1.7f}")

As we increase the number of trials the calculated value of $\pi$ becomes more accurate, so that seems good.

## Testing RNGs

- When developing an RNG, one needs to check if it's "good enough" for the intended use case. There is no definitive test, but there is a variety of checks that could be done.

- Example tests
  - Does it have a frequency distribution that is sensible? (in 1 or $N$ dimensions)
  - Are "gaps" between occurances correlated?
  - Are "runs" of numbers correlated?
  - Is the sequence length long enough for your application?
  - Are future numbers in the sequence easily predictible?
  - ...

- The current gold standard is [TestU01](https://simul.iro.umontreal.ca/testu01/tu01.html), which contains a battery of empirical tests.

## Let's implement some simple tests

In [ ]:
import scipy
def print_info(hist):
    n_entries = int(hist.sum())
    dim = len(h.shape)
    print(f"dim: {dim:1d}",end=", ")
    print(f"avg counts: {np.average(h):8.2f} sqrt of avg:{np.sqrt(np.average(h)):7.2f}",end=", ")
    print(f"std: {np.std(h):7.2f}",end=", ")
    print(f"chi2 pvalue: {scipy.stats.chisquare(np.resize(h.counts(),(n_entries,1))).pvalue[0]:2.8f}")

In [ ]:
from hist import Hist

n_rand = 1_000_000
rand_nums = my_lcg.randv(n_rand)
h = (
    Hist.new.Reg(50, 0, 1, name="x")
    .Int64()
)
h.fill(x=rand_nums)
print_info(h)

In [ ]:
h.plot()

# 3 common pitfalls (and how to avoid them)

## Pitfall #1: Using a bad RNG

- If you blindly use any RNG that you are handed, it is easy to not realize that you are using a bad RNG.

- Let's look at an infamous example, [IBM's RANDU](https://en.wikipedia.org/wiki/RANDU) generator. It was widely used in the 60s and 70s.

In [ ]:
randu = LCG(seed=1, multiplier=65539, addend=0, modulus=2**31)

## Let's perform the simple checks we implemented above.

In [ ]:
n_rand = 1_000_000
rand_nums = randu.randv(n_rand)
h = (
    Hist.new.Reg(50, 0, 1, name="x")
    .Int64()
)
h.fill(x=rand_nums)
print_info(h)

In [ ]:
h.plot()

So far, everything looks good.

## Now let's try to generate points in 3D

In [ ]:
n_rand = 3_000_000
rand_nums = randu.randv(n_rand)
h = (
    Hist
    .new
    .Reg(50, 0, 1, name="x")
    .Reg(50, 0, 1, name="y")
    .Reg(50, 0, 1, name="z")
    .Int64()
)
h.fill(
    x=rand_nums[0::3],
    y=rand_nums[1::3],
    z=rand_nums[2::3],
)
print_info(h)

The $p$-value is 0?! That doesn't look right. Let's plot the points we generated to see how they look.

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(projection="3d")

x=rand_nums[0:10000:3]
y=rand_nums[1:10001:3]
z=rand_nums[2:10002:3]

ax.scatter(x, y, z)

plt.show()

The points generated sit on a set of planes. So the coordinates are correlated!

## How to avoid this pitfall

Always use RNGs that have been well tested. Always use a library instead of trying to implement your own RNG.

## Pitfall #2: Abusing RNGs

- RNGs have limitations.

- PRNGs output a finite sequence that will eventually start repeating.

<center>
    <img src="images/rng_sequence.svg" style="height:80px;"/><br/>
</center>

- You should always make sure that only part of the available sequence is used.

<center>
    <img src="images/rng_good_subsequence.svg" style="height:80px;"/><br/>
</center>

- If you use the PRNG beyond the number of elements in it's output sequence, then you'll start reusing the sequence, which means that you are no longer using "random" numbers.

<center>
    <img src="images/rng_reused_subsequence.svg" style="height:120px;"/><br/>
</center>

Earlier, we computed $\pi$ as follows.

In [ ]:
calc_pi(my_rng, 1_000_000)

But if you look at how we defined the LCG above, the output sequence repeats every 714,025 elements. So we were actually reusing part of the sequence for this computation! If you take a closer look, you'll see that the average error and standard deviations we were getting were too small.

Modern PRNGs have extremely large periods, so this doesn't tend to be an issue. But they could still have limitations that affect your particular use case.

## How to avoid this pitfall

Know the limitations of the RNG you are using.

## Pitfall #3: Bad parallelization

- It's very common having to parallelize a computation that uses RNGs, and it's easy to overlook the steps needed to do it correctly.

- Let's run the tests of the $\pi$ computation we did above, but now running the iterations asynchronously.

In [ ]:
from concurrent.futures import ThreadPoolExecutor
powers=8
iterations=10

with ThreadPoolExecutor(max_workers=4) as executor:
    for decade in range(powers) :
        trials = 10**decade
        futures = []
        for i in range(iterations):
            lcg = LCG()
            futures.append(
                executor.submit(calc_pi, lcg, trials)
            )
        pis = np.array([f.result() for f in futures])     
        pi = np.average(pis)
        avg_err = np.average(np.abs(pis - np.pi))
        std = np.std(pis)
        print(f"{trials:8d} trials:", end=" ")
        print(f"pi = {pi:1.7f}", end=", ")
        print(f"avg err = {avg_err:1.7f}", end=", ")
        print(f"std = {std:1.7f}")

Hold on... all the standard deviations are zero, so that means that all iterations are returning the exact same result!

The issue is that all threads are using the same RNG with the same seed. So all threads are reusing the same subsequence.

<center>
    <img src="images/rng_bad_parallel.svg" style="height:120px;"/><br/>
</center>

A common thing people do is to give different seeds for each thread. But you still run the risk of having overlaps from the subsequences used by each thread.

<center>
    <img src="images/rng_bad_parallel2.svg" style="height:120px;"/><br/>
</center>

## How to avoid this pitfall

- Use RNGs that produce different sequences for each thread/process.
- Modern libraries have tools specifically designed for this.

- For example, `NumPy` is moving away from having a global RNG to having RNG objects that can then passed to threads.

In [ ]:
# The old way to use RNG
np.random.rand()

In [ ]:
# The new recommended way to use RNG
rng = np.random.default_rng()
rng.random()

These objects can spawn child RNG objects that are independent and can be used for parallelization.

In [ ]:
rng.spawn(4)

## Exercise 3

Using `NumPy` RNGs, resolve the pitfalls we taked about with the following code.

In [ ]:
from concurrent.futures import ThreadPoolExecutor
powers=8
iterations=10

def calc_pi(rng, num_trials: int) -> float:
    """Calculate PI with the given RNG"""
    x = rng.randv(num_trials)
    y = rng.randv(num_trials)
    in_circle = np.sum(x**2 + y**2 <= 1.0)
    return 4*(in_circle/num_trials)

with ThreadPoolExecutor(max_workers=4) as executor:
    for decade in range(powers) :
        trials = 10**decade
        futures = []
        for i in range(iterations):
            lcg = LCG()
            futures.append(
                executor.submit(calc_pi, lcg, trials)
            )
        pis = np.array([f.result() for f in futures])     
        pi = np.average(pis)
        avg_err = np.average(np.abs(pis - np.pi))
        std = np.std(pis)
        print(f"{trials:8d} trials:", end=" ")
        print(f"pi = {pi:1.7f}", end=", ")
        print(f"avg err = {avg_err:1.7f}", end=", ")
        print(f"std = {std:1.7f}")

<details>
<summary>
Answer (Don't look until you're done)
</summary>

<pre><code>
from concurrent.futures import ThreadPoolExecutor
powers=8
iterations=10

rng = np.random.default_rng()
rngs = rng.spawn(iterations)

def calc_pi(rng, num_trials: int) -> float:
    """Calculate PI with the given RNG"""
    x = rng.random(num_trials)
    y = rng.random(num_trials)
    in_circle = np.sum(x**2 + y**2 <= 1.0)
    return 4*(in_circle/num_trials)

with ThreadPoolExecutor(max_workers=4) as executor:
    for decade in range(powers) :
        trials = 10**decade
        futures = []
        for i in range(iterations):
            futures.append(
                executor.submit(calc_pi, rngs[i], trials)
            )
        pis = np.array([f.result() for f in futures])     
        pi = np.average(pis)
        avg_err = np.average(np.abs(pis - np.pi))
        std = np.std(pis)
        print(f"{trials:8d} trials:", end=" ")
        print(f"pi = {pi:1.7f}", end=", ")
        print(f"avg err = {avg_err:1.7f}", end=", ")
        print(f"std = {std:1.7f}")
</code></pre>
</details>

# Summary

- RNGs are very useful tools that are used everywhere.

- There are common pitfalls that are easy to miss, so you should be careful.

- Always use a library with good RNGs, and make sure to give independent RNGs to different threads/processes.